In [4]:
import cloudscraper
import pandas as pd

In [5]:
def scrape_team_stats(url, attrs):
    scraper = cloudscraper.create_scraper()
    html = scraper.get(url).text
    tables = pd.read_html(html, attrs=attrs, header=1)
    df = tables[0]
    df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

    return df

In [32]:
seasons = [
    "2020-2021",
    "2021-2022",
    "2022-2023",
    "2023-2024",
    "2024-2025",
    "2025-2026"
]

In [33]:
def clean_and_save_possession_stats(df, output_path):
    keep_cols = ["Squad", "Poss", "PrgR", "90s"]
    keep_cols = [c for c in keep_cols if c in df.columns]

    df = df[keep_cols]

    if all(col in df.columns for col in ["PrgR", "90s"]):
        df["PrgR_per90"] = (df["PrgR"] / df["90s"]).round(2)

    rename_cols = {
        "Poss": "Possession",
        "PrgR_per90": "Progressive_Passes_Received_Per90"
    }

    df = df.rename(columns=rename_cols)

    df = df.drop(columns=["PrgR", "90s"], errors="ignore")

    df.to_csv(output_path, index=False)

In [42]:
# POSSESSION FOR
base_url = "https://fbref.com/en/comps/9/{season}/possession/Premier-League-Stats"
attrs = {"id": "stats_squads_possession_for"}

for season in seasons:
    url = base_url.format(season=season)
    output_path = f"../CSV_files/possession_stats_for_{season.replace('-', '')}.csv"

    df = scrape_team_stats(url, attrs)
    clean_and_save_possession_stats(df, output_path)

/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/2600758676.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PrgR_per90"] = (df["PrgR"] / df["90s"]).round(2)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_h

In [36]:
# POSSESSION AGAINST
base_url_against = "https://fbref.com/en/comps/9/{season}/possession/Premier-League-Stats"
attrs_against = {"id": "stats_squads_possession_against"}

for season in seasons:
    url = base_url_against.format(season=season)
    output_path = f"../CSV_files/possession_stats_against_{season.replace('-', '')}.csv"

    df = scrape_team_stats(url, attrs_against)
    clean_and_save_possession_stats(df, output_path)

/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/2600758676.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PrgR_per90"] = (df["PrgR"] / df["90s"]).round(2)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_h

In [26]:
def clean_and_save_shooting_stats(df, output_path):
    keep_cols = ["Squad", "Sh/90", "SoT/90"]
    df = df[keep_cols]

    rename_cols = {
        "Sh/90": "Shots_Per_90",
        "SoT/90": "Shots_On_Target_Per_90"
    }

    df = df.rename(columns=rename_cols)
    df.to_csv(output_path, index=False)

In [37]:
# SHOOTING FOR
base_url_shoot_for = "https://fbref.com/en/comps/9/{season}/shooting/Premier-League-Stats"
attrs_shoot_for = {"id": "stats_squads_shooting_for"}

for season in seasons:
    url = base_url_shoot_for.format(season=season)
    output_path = f"../CSV_files/shooting_stats_for_{season.replace('-', '')}.csv"

    df = scrape_team_stats(url, attrs_shoot_for)
    clean_and_save_shooting_stats(df, output_path)

/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: Future

In [39]:
# SHOOTING  AGAINST
base_url_shoot_against = "https://fbref.com/en/comps/9/{season}/shooting/Premier-League-Stats"
attrs_shoot_against = {"id": "stats_squads_shooting_against"}

for season in seasons:
    url = base_url_shoot_against.format(season=season)
    output_path = f"../CSV_files/shooting_stats_against_{season.replace('-', '')}.csv"

    df = scrape_team_stats(url, attrs_shoot_against)
    clean_and_save_shooting_stats(df, output_path)


/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: Future

In [8]:
def clean_and_save_attacking_stats(df, output_path):
    keep_cols = ["Squad", "SCA90", "GCA90"]
    df = df[keep_cols]

    rename_cols = {
        "SCA90": "Shot_Creating_Actions_Per90",
        "GCA90": "Goal_Creating_Actions_Per90"
    }

    df = df.rename(columns=rename_cols)
    df.to_csv(output_path, index=False)

In [40]:
# ATTACKING FOR
base_url_att_for = "https://fbref.com/en/comps/9/{season}/gca/Premier-League-Stats"
attrs_attacking_for = {"id": "stats_squads_gca_for"}

for season in seasons:
    url = base_url_att_for.format(season=season)
    output_path = f"../CSV_files/attacking_stats_for_{season.replace('-', '')}.csv"

    df = scrape_team_stats(url, attrs_attacking_for)
    clean_and_save_attacking_stats(df, output_path)

/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: Future

In [41]:
# ATTACKING AGAINST
base_url_att_against = "https://fbref.com/en/comps/9/{season}/gca/Premier-League-Stats"
attrs_attacking_against = {"id": "stats_squads_gca_against"}

for season in seasons:
    url = base_url_att_against.format(season=season)
    output_path = f"../CSV_files/attacking_stats_against_{season.replace('-', '')}.csv"

    df = scrape_team_stats(url, attrs_attacking_against)
    clean_and_save_attacking_stats(df, output_path)

/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/jn/72qky7h53_v397xzn255p9fm0000gn/T/ipykernel_54033/1502654688.py:4: Future